# Imports

In [1]:
print("Importing libraries...")
import torch
from torch.cuda.amp import autocast, GradScaler
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import Dataset
import torchvision.transforms as T
import torchvision.transforms.functional as F
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.ops import MultiScaleRoIAlign
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import kagglehub
import shutil
from sklearn.model_selection import train_test_split
import json
from PIL import Image
import contextlib
import io
import time
import random
from torch.utils.data import Subset
from datetime import datetime
from tqdm import tqdm


Importing libraries...


/home/s3549852/.conda/envs/dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Functions

In [2]:
scaler = GradScaler()

# Define the dataset class
class HazmatDataset(Dataset):
    def __init__(self, data_dir, annotations_file, transforms=None):
        self.data_dir = data_dir
        self.transforms = transforms
        
        # Load annotations
        with open(annotations_file) as f:
            data = json.load(f)
        
        self.images = {img['id']: img for img in data['images']}
        self.annotations = data['annotations']
        
        # Create image_id to annotations mapping
        self.img_to_anns = {}
        for ann in self.annotations:
            img_id = ann['image_id']
            if img_id not in self.img_to_anns:
                self.img_to_anns[img_id] = []
            self.img_to_anns[img_id].append(ann)
        
        self.ids = list(self.images.keys())

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        img_info = self.images[img_id]
        
        # Load image
        img_path = os.path.join(self.data_dir, 'images', img_info['file_name'])
        img = Image.open(img_path).convert('RGB')
        
        # Get annotations
        anns = self.img_to_anns.get(img_id, [])
        
        boxes = []
        labels = []
        areas = []
        iscrowd = []
        
        for ann in anns:
            bbox = ann['bbox']
            # Convert [x, y, w, h] to [x1, y1, x2, y2]
            boxes.append([
                bbox[0],
                bbox[1],
                bbox[0] + bbox[2],
                bbox[1] + bbox[3]
            ])
            labels.append(ann['category_id'])
            areas.append(ann['area'])
            iscrowd.append(ann['iscrowd'])
        
        # Convert to tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        areas = torch.as_tensor(areas, dtype=torch.float32)
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)
        
        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([img_id]),
            'area': areas,
            'iscrowd': iscrowd
        }
        
        if self.transforms is not None:
            for transform in self.transforms:
                img, target = transform(img, target)
        
        return img, target

    def __len__(self):
        return len(self.ids)
    
class ToTensor(object):
    def __call__(self, image, target):
        # Convert PIL image to tensor
        image = F.to_tensor(image)
        return image, target

class RandomHorizontalFlip(object):
    def __init__(self, prob):
        self.prob = prob

    def __call__(self, image, target):
        if torch.rand(1) < self.prob:
            height, width = image.shape[-2:]
            image = F.hflip(image)
            # Flip bounding boxes
            bbox = target["boxes"]
            bbox[:, [0, 2]] = width - bbox[:, [2, 0]]  # Flip x-coordinates
            target["boxes"] = bbox
        return image, target

def get_transform(train):
    transforms = []
    # Convert PIL image to tensor
    transforms.append(ToTensor())
    if train:
        # Add training augmentations here if needed
        transforms.append(RandomHorizontalFlip(0.5))
    return transforms

def collate_fn(batch):
    return tuple(zip(*batch))

def train_one_epoch(model, optimizer, data_loader, device, scaler):
    model.train()
    total_loss = 0
    total_classifier_loss = 0
    total_box_reg_loss = 0
    total_objectness_loss = 0
    total_rpn_box_reg_loss = 0

    # Voeg tqdm toe om de voortgang te tonen
    progress_bar = tqdm(data_loader, desc="Training", leave=True)
    
    for images, targets in progress_bar:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Wrap the forward pass in autocast
        with autocast():
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        # Scale the loss and call backward
        scaler.scale(losses).backward()
        # Unscales the gradients and calls or skips optimizer.step()
        scaler.step(optimizer)
        # Updates the scale for next iteration
        scaler.update()

        # Bereken de totalen
        total_loss += losses.item()
        total_classifier_loss += loss_dict['loss_classifier'].item()
        total_box_reg_loss += loss_dict['loss_box_reg'].item()
        total_objectness_loss += loss_dict['loss_objectness'].item()
        total_rpn_box_reg_loss += loss_dict['loss_rpn_box_reg'].item()

        # Update tqdm-balk
        progress_bar.set_postfix({
            "Loss": f"{losses.item():.4f}",
            "Classifier": f"{loss_dict['loss_classifier'].item():.4f}",
            "BoxReg": f"{loss_dict['loss_box_reg'].item():.4f}",
        })

    avg_loss = total_loss / len(data_loader)
    avg_classifier_loss = total_classifier_loss / len(data_loader)
    avg_box_reg_loss = total_box_reg_loss / len(data_loader)
    avg_objectness_loss = total_objectness_loss / len(data_loader)
    avg_rpn_box_reg_loss = total_rpn_box_reg_loss / len(data_loader)

    return avg_loss, avg_classifier_loss, avg_box_reg_loss, avg_objectness_loss, avg_rpn_box_reg_loss



# Load ground truth annotations
coco_val = COCO('data/data_faster_rcnn/val/annotations/instances_val.json')

# Prepare predictions in COCO format
# Assuming you have a function to convert model outputs to COCO format
# Conversion to COCO Format
def convert_to_coco_format(outputs, image_ids):
    coco_results = []
    for output, image_id in zip(outputs, image_ids):
        boxes = output['boxes'].cpu().numpy()
        scores = output['scores'].cpu().numpy()
        labels = output['labels'].cpu().numpy()
        
        for box, score, label in zip(boxes, scores, labels):
            coco_results.append({
                'image_id': image_id,
                'category_id': int(label),
                'bbox': [box[0], box[1], box[2] - box[0], box[3] - box[1]],
                'score': float(score)
            })
    return coco_results

# Validation Function
def validate(model, data_loader, coco_gt, device):
    model.eval()
    results = []

    # Add tqdm
    progress_bar = tqdm(data_loader, desc="Validation", leave=True)

    with torch.no_grad():
        for images, targets in progress_bar:
            images = list(image.to(device) for image in images)
            outputs = model(images)
            
            image_ids = [target['image_id'].item() for target in targets]
            coco_results = convert_to_coco_format(outputs, image_ids)
            results.extend(coco_results)

            # Update tqdm-bar
            progress_bar.set_postfix({"Processed": len(results)})

    if not results:
        print("No predictions generated. Skipping evaluation.")
        return [0.0] * 6  # Return dummy metrics for empty results

    # Suppress COCOeval output
    with contextlib.redirect_stdout(io.StringIO()):
        coco_dt = coco_gt.loadRes(results)
        coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()

    return coco_eval.stats


# Custom backbone to return a dictionary of feature maps
class BackboneWithChannels(torch.nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
    def forward(self, x):
        x = self.backbone(x)
        return {'0': x}
    
# Function to create a subset of the dataset
def create_subset(dataset, percentage):
    """
    Create a subset of the dataset based on the given percentage.
    
    Parameters:
    - dataset: The full dataset.
    - percentage: The fraction of the dataset to use (value between 0.0 and 1.0).
    
    Returns:
    - subset: A subset of the dataset containing the specified percentage of data.
    """
    if not (0.0 < percentage <= 1.0):
        raise ValueError("Percentage must be between 0.0 and 1.0.")
    
    # Determine the subset size
    total_samples = len(dataset)
    subset_size = int(total_samples * percentage)
    
    # Shuffle and select a random subset of indices
    indices = list(range(total_samples))
    random.shuffle(indices)
    subset_indices = indices[:subset_size]
    
    return Subset(dataset, subset_indices)

def create_directory(base_path="data/models"):
    """
    Create a directory inside the base path named 'faster-rcnn-finetuned-{date}' 
    to store models and logs. The name includes the current date and time in the format 'DD-MM-YYYY HH:MM:SS'.

    Parameters:
    - base_path (str): Base directory where the new directory will be created.

    Returns:
    - directory_path (str): Full path to the created directory.
    """
    # Get the current date and time
    current_time = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
    
    # Define the full directory path
    directory_name = f"faster-rcnn-finetuned-{current_time}"
    directory_path = os.path.join(base_path, directory_name)
    
    # Create the directory
    os.makedirs(directory_path, exist_ok=True)
    
    print(f"Directory created: {directory_path}")
    return directory_path

def train_model(directory, model, optimizer, train_loader, device, train_metrics_list, best_val_map, lr_scheduler, val_loader, coco_val, scaler, epoch):
    
    epoch+=1
    # Start the timer
    start_time = time.time()
    
    # Train for one epoch
    train_loss, train_classifier_loss, train_box_reg_loss, train_objectness_loss, train_rpn_box_reg_loss = train_one_epoch(
        model, optimizer, train_loader, device, scaler)
    
    # Validate and get all COCO-metrics
    val_metrics = validate(model, val_loader, coco_val, device)
    val_map = val_metrics[0]  # mAP@IoU=0.50:0.95
    
    # Stop the timer
    end_time = time.time()
    elapsed_time = end_time - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    
    # Obtain the current learning rate
    current_lr = optimizer.param_groups[0]['lr']
    
    # Prepare data for logging
    data = {
        "epoch": epoch,
        "time_elapsed": (int(minutes), int(seconds)),
        "learning_rate": current_lr,
        "train_loss": train_loss,
        "classifier_loss": train_classifier_loss,
        "box_reg_loss": train_box_reg_loss,
        "objectness_loss": train_objectness_loss,
        "rpn_box_reg_loss": train_rpn_box_reg_loss,
        "val_metrics": val_metrics
    }
    
    # Append current epoch data to metrics list
    train_metrics_list.append(data)
    
    # Print summary for this epoch
    print(f"📊 Epoch {epoch} | ⏳ Time: {int(minutes)}m {int(seconds)}s | 🔄 LR: {current_lr:.6f}")
    print(f"📉 Train Loss: {train_loss:.4f} | 🎯 Classifier: {train_classifier_loss:.4f} | 📦 Box Reg: {train_box_reg_loss:.4f}")
    print(f"🔍 Objectness: {train_objectness_loss:.4f} | 🗂️ RPN Box Reg: {train_rpn_box_reg_loss:.4f}")
    print(f"🧪 mAP | 🟢 mAP@IoU=0.50:0.95: {val_metrics[0]:.4f} | 🔵 mAP@IoU=0.50: {val_metrics[1]:.4f} | 🟣 mAP@IoU=0.75: {val_metrics[2]:.4f}")
    print(f"📏 Small mAP: {val_metrics[3]:.4f} | 📐 Medium mAP: {val_metrics[4]:.4f} | 📏 Large mAP: {val_metrics[5]:.4f}")
    
    # Save epoch data to a log file
    save_epoch_data(directory, data)
    
    # Update learning rate
    lr_scheduler.step()
    
    # Save the latest checkpoint with all metrics
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'val_map': val_map,
        'train_metrics_list': train_metrics_list  # Save all metrics
    }
    torch.save(checkpoint, os.path.join(directory, "latest_model.pth"))
    
    # Save the best model if the val_map is the highest so far
    if val_map > best_val_map:
        best_val_map = val_map
        torch.save(checkpoint, os.path.join(directory, "best_model.pth"))
    
    return best_val_map
        


def save_epoch_data(directory, data):
    """
    Save training statistics for each epoch in a text file.

    Parameters:
    - directory (str): Path to the directory.
    - data (dict): Contains data on metrics such as epoch, losses, and validation metrics.
    """
    log_file_path = os.path.join(directory, "training_log.txt")
    
    with open(log_file_path, "a") as log_file:
        log_file.write(f"📊 Epoch {data['epoch']} | ⏳ Time: {data['time_elapsed'][0]}m {data['time_elapsed'][1]}s | 🔄 LR: {data['learning_rate']:.6f}\n")
        log_file.write(f"📉 Train Loss: {data['train_loss']:.4f} | 🎯 Classifier: {data['classifier_loss']:.4f} | 📦 Box Reg: {data['box_reg_loss']:.4f}\n")
        log_file.write(f"🔍 Objectness: {data['objectness_loss']:.4f} | 🗂️ RPN Box Reg: {data['rpn_box_reg_loss']:.4f}\n")
        log_file.write(f"🧪 Validation Metrics | 🟢 mAP@IoU=0.50:0.95: {data['val_metrics'][0]:.4f} | 🔵 mAP@IoU=0.50: {data['val_metrics'][1]:.4f} | 🟣 mAP@IoU=0.75: {data['val_metrics'][2]:.4f}\n")
        log_file.write(f"📏 Small mAP: {data['val_metrics'][3]:.4f} | 📐 Medium mAP: {data['val_metrics'][4]:.4f} | 📏 Large mAP: {data['val_metrics'][5]:.4f}\n")
        log_file.write("\n")

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


/tmp/ipykernel_1583115/3664114777.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


# Model initialisation

In [3]:
from collections import OrderedDict
device = torch.device('cuda:0')
print(f"Training model on {device}")

# Create datasets
train_dataset = HazmatDataset(
    data_dir='data/data_faster_rcnn/train',
    annotations_file='data/data_faster_rcnn/train/annotations/instances_train.json',
    transforms=get_transform(train=True)
)

val_dataset = HazmatDataset(
    data_dir='data/data_faster_rcnn/val',
    annotations_file='data/data_faster_rcnn/val/annotations/instances_val.json',
    transforms=get_transform(train=False)
)

# Set the percentage of the training dataset to use (e.g. 0.x to 1)
train_percentage = 1

# Create a subset of the training dataset
train_dataset_subset = create_subset(train_dataset, train_percentage)

# Set the percentage of the val dataset to use (e.g. 0.x to 1)
val_percentage = 1

# Create a subset of the training dataset
val_dataset_subset = create_subset(val_dataset, val_percentage)

# amount of cpu cores
workers = 2

# Create data loaders
train_loader = DataLoader(
    train_dataset_subset,
    batch_size=16,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=workers,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset_subset,
    batch_size=16,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=workers,
    pin_memory=True
)

# Initialize model
num_classes = 2  # hazmat code and background

# Create ResNet-101 backbone with FPN
backbone = resnet_fpn_backbone('resnet101', pretrained=True)

# Define anchor generator for FPN
anchor_generator = AnchorGenerator(
    sizes=((32,), (64,), (128,), (256,), (512,)),
    aspect_ratios=((0.5, 1.0, 2.0),) * 5
)

# Multi-scale RoI pooling for FPN
roi_pooler = MultiScaleRoIAlign(
    featmap_names=['0', '1', '2', '3', '4'],
    output_size=7,
    sampling_ratio=2
)

print("initializing model...")
# Initialize Faster R-CNN with ResNet-101-FPN
model = FasterRCNN(
    backbone=backbone,
    num_classes=num_classes,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)

# Move model to device
model.to(device)

Training model on cuda:0


/home/s3549852/.conda/envs/dev/lib/python3.12/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/s3549852/.conda/envs/dev/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/s3549852/.conda/envs/dev/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


initializing model...


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [4]:
import subprocess

def show_gpu_usage():
    result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE)
    print(result.stdout.decode())

show_gpu_usage()
# Check how many GPUs are available
num_gpus = torch.cuda.device_count()
print(f"Number of GPUs available: {num_gpus}")

Tue Jan 14 11:38:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.76                 Driver Version: 550.76         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A40                     On  |   00000000:17:00.0 Off |                    0 |
|  0%   69C    P0            229W /  300W |    9121MiB /  46068MiB |     43%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# !kill -9 7710

# Training

In [ ]:
# Initialize optimizer and scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Training loop
num_epochs = 23
train_metrics_map = []
best_val_map = float('-inf')

print("Starting training...")

# Create directory to store models and logs
directory_finetuned_model = create_directory()


for epoch in range(num_epochs):
    best_val_map = train_model(
        directory=directory_finetuned_model, 
        model=model, optimizer=optimizer, train_loader=train_loader, device=device, 
        train_metrics_list=train_metrics_map, best_val_map=best_val_map, lr_scheduler=lr_scheduler, 
        val_loader=val_loader, coco_val=coco_val, scaler=scaler, epoch=epoch
    )



Starting training...
Directory created: data/models/faster-rcnn-finetuned-14-01-2025 11:39:00


Training:   0%|          | 0/478 [00:00<?, ?it/s]/tmp/ipykernel_1583115/3664114777.py:126: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Validation: 100%|██████████| 90/90 [03:36<00:00,  2.41s/it, Processed=9739]


📊 Epoch 1 | ⏳ Time: 26m 7s | 🔄 LR: 0.005000
📉 Train Loss: 0.1701 | 🎯 Classifier: 0.0712 | 📦 Box Reg: 0.0666
🔍 Objectness: 0.0289 | 🗂️ RPN Box Reg: 0.0033
🧪 mAP | 🟢 mAP@IoU=0.50:0.95: 0.3037 | 🔵 mAP@IoU=0.50: 0.5762 | 🟣 mAP@IoU=0.75: 0.2738
📏 Small mAP: -1.0000 | 📐 Medium mAP: 0.2000 | 📏 Large mAP: 0.3067


Validation: 100%|██████████| 90/90 [03:22<00:00,  2.25s/it, Processed=15272]


📊 Epoch 2 | ⏳ Time: 24m 15s | 🔄 LR: 0.005000
📉 Train Loss: 0.1415 | 🎯 Classifier: 0.0610 | 📦 Box Reg: 0.0684
🔍 Objectness: 0.0097 | 🗂️ RPN Box Reg: 0.0023
🧪 mAP | 🟢 mAP@IoU=0.50:0.95: 0.3513 | 🔵 mAP@IoU=0.50: 0.6884 | 🟣 mAP@IoU=0.75: 0.3182
📏 Small mAP: -1.0000 | 📐 Medium mAP: 0.2005 | 📏 Large mAP: 0.3552


...

Validation: 100%|██████████| 90/90 [03:26<00:00,  2.30s/it, Processed=6448]


📊 Epoch 14 | ⏳ Time: 24m 1s | 🔄 LR: 0.000001
📉 Train Loss: 0.0945 | 🎯 Classifier: 0.0369 | 📦 Box Reg: 0.0543
🔍 Objectness: 0.0020 | 🗂️ RPN Box Reg: 0.0012
🧪 mAP | 🟢 mAP@IoU=0.50:0.95: 0.5082 | 🔵 mAP@IoU=0.50: 0.8533 | 🟣 mAP@IoU=0.75: 0.5128
📏 Small mAP: -1.0000 | 📐 Medium mAP: 0.4319 | 📏 Large mAP: 0.5116


Validation: 100%|██████████| 90/90 [03:24<00:00,  2.27s/it, Processed=6441]


📊 Epoch 15 | ⏳ Time: 24m 3s | 🔄 LR: 0.000001
📉 Train Loss: 0.0938 | 🎯 Classifier: 0.0367 | 📦 Box Reg: 0.0538
🔍 Objectness: 0.0020 | 🗂️ RPN Box Reg: 0.0012
🧪 mAP | 🟢 mAP@IoU=0.50:0.95: 0.5083 | 🔵 mAP@IoU=0.50: 0.8533 | 🟣 mAP@IoU=0.75: 0.5143
📏 Small mAP: -1.0000 | 📐 Medium mAP: 0.4319 | 📏 Large mAP: 0.5120


Training:  83%|████████▎ | 395/478 [17:14<03:35,  2.59s/it, Loss=0.0981, Classifier=0.0400, BoxReg=0.0551]

# Evaluation

In [ ]:
# Load the model
device = torch.device('cuda')
model_path = os.path.join(directory_finetuned_model, 'best_model.pth')
checkpoint = torch.load(model_path, map_location=device)
val_map = checkpoint['val_map']
epoch = checkpoint['epoch']
#latest
latest_model_path = os.path.join(directory_finetuned_model, 'latest_model.pth')
checkpoint_latest = torch.load(latest_model_path, map_location=device)
val_map_latest = checkpoint_latest['val_map']
epoch_latest = checkpoint_latest['epoch']

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # Set the model to evaluation mode

print(f"Validation mAP best model: {val_map:.4f}")
print(f"Epoch best model: {epoch}")

print(f"Validation mAP latest model: {val_map_latest:.4f}")
print(f"Epoch latest model: {epoch_latest}")


In [ ]:
import matplotlib.pyplot as plt

def plot_metrics(checkpoint_path, title="Training and Validation Metrics over Epochs"):
    """
    Plot training and validation metrics from a given model checkpoint.
    
    Parameters:
    - checkpoint_path (str): Path to the model checkpoint file (e.g., 'latest_model.pth').
    - title (str): Title for the plot.
    """
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    train_metrics_list = checkpoint['train_metrics_list']
    
    # Extract metrics per epoch
    epochs = [data['epoch'] for data in train_metrics_list]
    train_loss_list = [data['train_loss'] for data in train_metrics_list]
    classifier_loss_list = [data['classifier_loss'] for data in train_metrics_list]
    box_reg_loss_list = [data['box_reg_loss'] for data in train_metrics_list]
    objectness_loss_list = [data['objectness_loss'] for data in train_metrics_list]
    rpn_box_reg_loss_list = [data['rpn_box_reg_loss'] for data in train_metrics_list]

    # Extract validation mAP metrics
    val_map_list = [data['val_metrics'][0] for data in train_metrics_list]  # mAP@IoU=0.50:0.95
    val_map_50_list = [data['val_metrics'][1] for data in train_metrics_list]  # mAP@IoU=0.50
    val_map_75_list = [data['val_metrics'][2] for data in train_metrics_list]  # mAP@IoU=0.75
    val_map_small_list = [data['val_metrics'][3] for data in train_metrics_list]  # Small mAP
    val_map_medium_list = [data['val_metrics'][4] for data in train_metrics_list]  # Medium mAP
    val_map_large_list = [data['val_metrics'][5] for data in train_metrics_list]  # Large mAP

    # Initialize the plot
    plt.figure(figsize=(14, 10))

    # Plot training losses
    plt.plot(epochs, train_loss_list, label='Training Loss', marker='o')
    #     plt.plot(epochs, classifier_loss_list, label='Classifier Loss', marker='o')
    #     plt.plot(epochs, box_reg_loss_list, label='Box Regression Loss', marker='o')
    #     plt.plot(epochs, objectness_loss_list, label='Objectness Loss', marker='o')
    #     plt.plot(epochs, rpn_box_reg_loss_list, label='RPN Box Regression Loss', marker='o')

    # Plot validation mAP metrics
    plt.plot(epochs, val_map_list, label='Validation mAP (IoU=0.50:0.95)', linestyle='--', marker='x')
    plt.plot(epochs, val_map_50_list, label='Validation mAP (IoU=0.50)', linestyle='--', marker='x')
    plt.plot(epochs, val_map_75_list, label='Validation mAP (IoU=0.75)', linestyle='--', marker='x')
    plt.plot(epochs, val_map_small_list, label='Validation mAP (Small)', linestyle='--', marker='x')
    plt.plot(epochs, val_map_medium_list, label='Validation mAP (Medium)', linestyle='--', marker='x')
    plt.plot(epochs, val_map_large_list, label='Validation mAP (Large)', linestyle='--', marker='x')

    # Set x-axis ticks to start from 1
    plt.xticks(range(1, len(epochs) + 1))

    # Set plot details
    plt.xlabel('Epoch')
    plt.ylabel('Metric Value')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Load latest model checkpoint
latest_model_path = os.path.join(directory_finetuned_model, 'latest_model.pth')
plot_metrics(latest_model_path, "Training and validation over epochs")

In [ ]:
def load_image(image_path, transforms=None):
    image = Image.open(image_path).convert('RGB')
    if transforms:
        for transform in transforms:
            image, _ = transform(image, target=None)  # No target during inference
    return image

# Define preprocessing transforms
test_transforms = get_transform(train=False)

# Load the image
image_path = 'images/hazard_plate.jpg'  # Replace with your image path
image = load_image(image_path, transforms=test_transforms)
image = image.to(device)
# Wrap the image in a list as the model expects a batch
with torch.no_grad():
    predictions = model([image])

In [ ]:
def draw_predictions(image, predictions, threshold=0.5, classes=['background', 'hazmat']):
    # Convert image from tensor to numpy array
    image = image.cpu().permute(1, 2, 0).numpy()
    image = np.clip(image * 255, 0, 255).astype(np.uint8)
    
    boxes = predictions[0]['boxes'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    
    # Filter predictions based on confidence threshold
    keep = scores >= threshold
    boxes = boxes[keep]
    labels = labels[keep]
    scores = scores[keep]
    
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(image)
    
    for box, label, score in zip(boxes, labels, scores):
        if label == 1:  # Only plot hazmat codes
            x1, y1, x2, y2 = box
            rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, color='blue', linewidth=2)
            ax.add_patch(rect)
            label_name = classes[label]
            ax.text(x1, y1, f'{label_name}: {score:.2f}', color='white', backgroundcolor='blue', fontsize=12)
    
    plt.show()

In [ ]:
def predict_image(image_path, threshold=0.5):
    # List of class names
    classes = ['background', 'hazmat']
    
    # Load the image
    image = load_image(image_path, transforms=test_transforms)
    image = image.to(device)
    
    # Wrap the image in a list as the model expects a batch
    with torch.no_grad():
        predictions = model([image])
    
    # Filter predictions based on threshold
    boxes = predictions[0]['boxes'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    
    # Apply threshold filter
    keep = scores >= threshold
    boxes = boxes[keep]
    labels = labels[keep]
    scores = scores[keep]
    
    # Print the predictions
    if len(boxes) == 0:
        print("No predictions meet the threshold.")
    else:
        print("Predictions:")
        for label, score in zip(labels, scores):
            class_name = classes[label]
            print(f"  {class_name}: {score:.2f}")
        # Display the predictions
        draw_predictions(image, predictions, threshold=threshold, classes=classes)

In [ ]:
predict_image('data/data_faster_rcnn/val/images/1690281365_00595.jpg', threshold=0.29)
predict_image('images/hazard_plate.jpg', threshold=0)
predict_image('images/un_numbers_test/1.webp', threshold=0)
predict_image('images/un_numbers_test/2.jpg', threshold=0)
predict_image('images/un_numbers_test/3.jpg', threshold=0)
predict_image('images/un_numbers_test/4.jpg', threshold=0)
predict_image('images/un_numbers_test/6.webp', threshold=0)
predict_image('images/un_numbers_test/7.jpg', threshold=0)


In [ ]:
# loop trhough all images from test and predict

test_images_path = "data/data_faster_rcnn/test/images"

# frames available
test_images_path_list = os.listdir(test_images_path)
import random
random.shuffle(test_images_path_list)

# Predict on the first 30 images
for count, image_name in enumerate(test_images_path_list[:20]):
    image_path = os.path.join(test_images_path, image_name)
    predict_image(image_path, threshold=0.4)